# Single Month Dataset


In [9]:
import requests
import pandas as pd
import re
import requests
import warnings
import urllib.parse

warnings.simplefilter("ignore", category=UserWarning)

In [10]:
base_endpoint = 'https://opendata.nhsbsa.net/api/3/action/'
package_list_method = 'package_list'     # List of data-sets in the portal
package_show_method = 'package_show?id=' # List all resources of a data-set
action_method = 'datastore_search_sql?'  # SQL action method

In [11]:
datasets_response = requests.get(base_endpoint +  package_list_method).json()

In [12]:
dataset_id = "english-prescribing-data-epd"

In [13]:
resource_name = 'EPD_202411' # For EPD resources are named EPD_YYYYMM - Nov-24

In [14]:
#Read practice codes from CSV file
# Metformin  - 0601022B0
# Exenatide - 0601023Y0
# Dapagliflozin - 0601023AG
practice_csv_file = "csv_files/practice_codes.csv"  # Change to your actual file name
df_practice = pd.read_csv(practice_csv_file,header=None, names=['PRACTICE_CODE'])
df_practice.count()
# Ensure the column name matches your CSV file
practice_code_list = df_practice['PRACTICE_CODE'].dropna().astype(str).unique().tolist()
#Convert the list into a string of quoted values for SQL query
practice_code_str = ", ".join([f"'{code}'" for code in practice_code_list])

In [15]:
#Read bnf checmical substance codes from CSV file
bnf_sub_code_csv_file = "csv_files/bnf_chemical_sub_code.csv"  # Change to your actual file name
df_bnf_substance = pd.read_csv(bnf_sub_code_csv_file,header=None, names=['BNF_CHEMICAL_SUBSTANCE'])
df_bnf_substance.count()
# Ensure the column name matches your CSV file
bnf_sub_code_list = df_bnf_substance['BNF_CHEMICAL_SUBSTANCE'].dropna().astype(str).unique().tolist()
#Convert the list into a string of quoted values for SQL query
bnf_chemical_substance_str = ", ".join([f"'{code}'" for code in bnf_sub_code_list])

In [16]:
single_month_query = "SELECT * " \
                     f"FROM `{resource_name}` " \
                     f"WHERE practice_code in ({practice_code_str}) " \
                     f"AND bnf_chemical_substance in ({bnf_chemical_substance_str})"

In [19]:
single_month_query

"SELECT * FROM `EPD_202411` WHERE practice_code in ('F84031', 'F84087', 'F84030', 'Y03023', 'F84055', 'F84062', 'F84079', 'F84114', 'F84122', 'F84081', 'F84718', 'F84083', 'F84118', 'F84698', 'F84747', 'F84714', 'F84656', 'F84012', 'F84051', 'F84710', 'F84016', 'F84025', 'F84054', 'F84123', 'F84034', 'F84044', 'F84696', 'F84039', 'F84733', 'F84647', 'F84731') AND bnf_chemical_substance in ('0601022B0', '0601023Y0', '0601023AG')"

In [17]:
single_month_api_call = f"{base_endpoint}" \
                        f"{action_method}" \
                        "resource_id=" \
                        f"{resource_name}" \
                        "&" \
                        "sql=" \
                        f"{urllib.parse.quote(single_month_query)}" # Encode spaces in the url

In [18]:
single_month_response = requests.get(single_month_api_call).json()

In [20]:
single_month_df  = pd.json_normalize(single_month_response['result']['result']['records'])

In [16]:
single_month_df.count()

BNF_CODE                        1216
TOTAL_QUANTITY                  1216
POSTCODE                        1216
YEAR_MONTH                      1216
UNIDENTIFIED                    1216
PRACTICE_NAME                   1216
ICB_NAME                        1216
BNF_CHAPTER_PLUS_CODE           1216
ICB_CODE                        1216
ACTUAL_COST                     1216
QUANTITY                        1216
REGIONAL_OFFICE_CODE            1216
ITEMS                           1216
ADDRESS_4                       1216
ADDRESS_1                       1216
ADDRESS_2                       1216
ADDRESS_3                       1216
BNF_CHEMICAL_SUBSTANCE          1216
ADQUSAGE                        1216
PCO_CODE                        1216
REGIONAL_OFFICE_NAME            1216
NIC                             1216
CHEMICAL_SUBSTANCE_BNF_DESCR    1216
PRACTICE_CODE                   1216
PCO_NAME                        1216
BNF_DESCRIPTION                 1216
dtype: int64

In [21]:
single_month_df.to_csv('download/single_month.csv')

# Five Years Data

In [22]:
metadata_repsonse  = requests.get(f"{base_endpoint}" \
                                  f"{package_show_method}" \
                                  f"{dataset_id}").json()

In [25]:
resources_table  = pd.json_normalize(metadata_repsonse['result']['resources'])

In [26]:
import datetime
# Define dynamic start and end years (you can set any range)
start_year = 2020  # Change as needed
end_year = 2025    # Change as needed

# Ensure start year is not greater than end year
if start_year > end_year:
    raise ValueError("Start year must be less than or equal to the end year.")

# Convert start and end years into a regex pattern
year_pattern = f"({'|'.join(str(year) for year in range(start_year, end_year + 1))})"

# Filter resources dynamically based on the year range
resource_name_list = resources_table[resources_table['name'].str.contains(year_pattern, regex=True)]['name']


In [29]:
resource_name_list

72     EPD_202001
73     EPD_202002
74     EPD_202003
75     EPD_202004
76     EPD_202005
77     EPD_202006
78     EPD_202007
79     EPD_202008
80     EPD_202009
81     EPD_202010
82     EPD_202011
83     EPD_202012
84     EPD_202101
85     EPD_202102
86     EPD_202103
87     EPD_202104
88     EPD_202105
89     EPD_202106
90     EPD_202107
91     EPD_202108
92     EPD_202109
93     EPD_202110
94     EPD_202111
95     EPD_202112
96     EPD_202201
97     EPD_202202
98     EPD_202203
99     EPD_202204
100    EPD_202205
101    EPD_202206
102    EPD_202207
103    EPD_202208
104    EPD_202209
105    EPD_202210
106    EPD_202211
107    EPD_202212
108    EPD_202301
109    EPD_202302
110    EPD_202303
111    EPD_202304
112    EPD_202305
113    EPD_202306
114    EPD_202307
115    EPD_202308
116    EPD_202309
117    EPD_202310
118    EPD_202311
119    EPD_202312
120    EPD_202401
121    EPD_202402
122    EPD_202403
123    EPD_202404
124    EPD_202405
125    EPD_202406
126    EPD_202407
127    EPD

In [30]:
import time
import json
for month in resource_name_list:

    # Build temporary SQL query
    tmp_query = "SELECT * " \
                     f"FROM `{resource_name}` " \
                     f"WHERE practice_code in ({practice_code_str}) " \
                     f"AND bnf_chemical_substance in ({bnf_chemical_substance_str})"

    # Build temporary API call
    tmp_api_call  = f"{base_endpoint}" \
                    f"{action_method}" \
                    "resource_id=" \
                    f"{month}" \
                    "&" \
                    "sql=" \
                    f"{urllib.parse.quote(tmp_query)}" # Encode spaces in the url
    # Data storage
    all_data = []
    # # Grab the response JSON as a temporary list
    # tmp_response = requests.get(tmp_api_call).json()
    # Data storage
all_data = []

# Iterate over each month in resource_name_list
for month in resource_name_list:
    print(f"Fetching data for {month}...")

    # Build SQL query
    tmp_query = f"""
        SELECT *
        FROM `{month}`
        WHERE practice_code IN ({practice_code_str})
        AND bnf_chemical_substance IN ({bnf_chemical_substance_str})
    """

    # Build API call
    tmp_api_call = f"{base_endpoint}{action_method}resource_id={month}&sql={urllib.parse.quote(tmp_query)}"

    try:
        # Request data
        tmp_response = requests.get(tmp_api_call, timeout=60)

        if tmp_response.status_code == 200:
            try:
                response_json = tmp_response.json()  # Parse JSON response

                # **Handle nested "result" structure**
                if isinstance(response_json, dict) and "result" in response_json:
                    nested_result = response_json.get("result", {})

                    if isinstance(nested_result, dict) and "result" in nested_result:
                        final_result = nested_result.get("result", {})

                        if isinstance(final_result, dict) and "records" in final_result:
                            records = final_result.get("records", [])

                            if records:
                                all_data.extend(records)  # Append data
                                print(f"✔ {len(records)} records fetched for {month}.")
                            else:
                                print(f"⚠ No records found for {month}.")
                        else:
                            print(f"⚠ Missing 'records' key in JSON response for {month}. Skipping...")

                    else:
                        print(f"⚠ Missing second-level 'result' key in JSON response for {month}. Skipping...")

                else:
                    print(f"⚠ Invalid JSON structure for {month}. Skipping...")

            except json.JSONDecodeError:
                print(f"❌ Failed to decode JSON response for {month}. Skipping...")

        else:
            print(f"❌ API Error {tmp_response.status_code} for {month}: {tmp_response.text}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed for {month}: {e}")

    # Optional delay to prevent rate-limiting
    time.sleep(1)

# Convert data to DataFrame and save
if all_data:
    filtered_data = pd.DataFrame(all_data)
    filtered_data.to_csv("download/epd_filtered_data.csv", index=False)
    print("✅ Data retrieval complete. Saved as epd_filtered_data.csv")
else:
    print("❌ No data retrieved.")


Fetching data for EPD_202001...
✔ 998 records fetched for EPD_202001.
Fetching data for EPD_202002...
✔ 991 records fetched for EPD_202002.
Fetching data for EPD_202003...
✔ 1003 records fetched for EPD_202003.
Fetching data for EPD_202004...
✔ 964 records fetched for EPD_202004.
Fetching data for EPD_202005...
✔ 948 records fetched for EPD_202005.
Fetching data for EPD_202006...
✔ 966 records fetched for EPD_202006.
Fetching data for EPD_202007...
✔ 1016 records fetched for EPD_202007.
Fetching data for EPD_202008...
✔ 955 records fetched for EPD_202008.
Fetching data for EPD_202009...
✔ 1003 records fetched for EPD_202009.
Fetching data for EPD_202010...
✔ 1010 records fetched for EPD_202010.
Fetching data for EPD_202011...
✔ 989 records fetched for EPD_202011.
Fetching data for EPD_202012...
✔ 1001 records fetched for EPD_202012.
Fetching data for EPD_202101...
✔ 1006 records fetched for EPD_202101.
Fetching data for EPD_202102...
✔ 983 records fetched for EPD_202102.
Fetching data 

In [31]:
# Function to get latitude and longitude from postcode
def get_lat_lon(postcode):
    url = f"https://api.postcodes.io/postcodes/{postcode}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data['result']['latitude'], data['result']['longitude']
    else:
        return None, None  # Return None if API call fails


# Get unique postcodes
unique_postcodes = filtered_data['POSTCODE'].unique()

# Create a dictionary to store lat/lon for unique postcodes
postcode_lat_lon = {postcode: get_lat_lon(postcode) for postcode in unique_postcodes}

# Map the lat/lon values back to the DataFrame
filtered_data[['Latitude', 'Longitude']] = filtered_data['POSTCODE'].map(postcode_lat_lon).apply(pd.Series)

print(filtered_data[['POSTCODE','Latitude','Longitude']])
filtered_data.to_csv('download/EPD_Final.csv', index=False)

      POSTCODE   Latitude  Longitude
0       E1 0LS  51.513595  -0.050552
1       E1 1BU  51.520017  -0.057658
2       E1 1BU  51.520017  -0.057658
3       E1 2LP  51.514128  -0.062508
4       E1 2LP  51.514128  -0.062508
...        ...        ...        ...
67767   E3 5ED  51.532455  -0.031163
67768   E3 5TW  51.529785  -0.038875
67769  E14 0XA  51.512639  -0.005312
67770  E14 6PG  51.514211  -0.014336
67771  E14 8JH  51.500806  -0.025725

[67772 rows x 3 columns]
